In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from utils import *

In [16]:
import tensorflow as tf
import keras

In [33]:
def optimise_conv(hp):
	channels = 6
	length = 400
	
	input_shape = (length, channels)
	num_classes = 1

	input_layer = keras.layers.Input(input_shape)

	conv1 = keras.layers.Conv1D(filters=hp.Int("filters", min_value=32, max_value=128, step=32), kernel_size=hp.Int("kernel_size", min_value=4, max_value=10, step=2), padding="same")(input_layer)
	conv1 = keras.layers.BatchNormalization()(conv1)
	conv1 = keras.layers.ReLU()(conv1)

	conv2 = keras.layers.Conv1D(filters=hp.Int("filters", min_value=32, max_value=128, step=32), kernel_size=hp.Int("kernel_size", min_value=4, max_value=12, step=2), padding="same")(conv1)
	conv2 = keras.layers.BatchNormalization()(conv2)
	conv2 = keras.layers.ReLU()(conv2)

	conv3 = keras.layers.Conv1D(filters=hp.Int("filters", min_value=32, max_value=128, step=32), kernel_size=hp.Int("kernel_size", min_value=4, max_value=12, step=2), padding="same")(conv2)
	conv3 = keras.layers.BatchNormalization()(conv3)
	conv3 = keras.layers.ReLU()(conv3)

	gap = keras.layers.GlobalAveragePooling1D()(conv3)

	output_layer = keras.layers.Dense(num_classes, activation="sigmoid")(gap)

	model = keras.models.Model(inputs=input_layer, outputs=output_layer)

	model.compile(
		optimizer="adam",
		loss="binary_crossentropy",
		metrics=["accuracy"],
	)

	return model

In [29]:
# dataset specific
channels = 6
data = 400

print("Training on dataset with ", data, "datapoints...")
(df, X_conv, y) = get_data_conv("train", data, channels, extra="quat")
(df, X_test_conv, y_test) = get_data_conv("test", data, channels, extra="quat")

Training on dataset with  400 datapoints...
Loading training data (type= train datapoints= 400 )...
Reshaping input for convolution...
Loading training data (type= test datapoints= 400 )...
Reshaping input for convolution...


In [34]:
# train convolutional model
import keras_tuner

epochs = 60
batch_size = 32

print("Creating model...")
model = optimise_conv(keras_tuner.HyperParameters())

tuner = keras_tuner.RandomSearch(
    hypermodel=optimise_conv,
    objective="val_accuracy",
    executions_per_trial=1,
    overwrite=True,
    directory="./models/optimise/tuner",
    project_name="skiingai",
)

Creating model...


In [35]:
print("Searcing for best configs model...")
tuner.search(X_conv, y, epochs=5, validation_data=(X_test_conv, y_test))

Trial 10 Complete [00h 00m 40s]
val_accuracy: 0.6383700966835022

Best val_accuracy So Far: 0.7308998107910156
Total elapsed time: 00h 06m 24s


In [36]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400, 6)]          0         
                                                                 
 conv1d (Conv1D)             (None, 400, 96)           3552      
                                                                 
 batch_normalization (BatchN  (None, 400, 96)          384       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 400, 96)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 400, 96)           55392     
                                                                 
 batch_normalization_1 (Batc  (None, 400, 96)          384       
 hNormalization)                                             

In [37]:
result = best_model.evaluate(X_test_conv, y_test)
print("Accuracy of model", result[1])

37/37 [==============================] - 1s 16ms/step - loss: 0.5280 - accuracy: 0.7309
Accuracy of model 0.7308998107910156


In [39]:
best_model.save("./models/optimise/best_tuned.keras")

In [40]:
# Evaluate best_tuned.keras convolutional model trained on 400 datapoints

model = tf.keras.models.load_model('./models/optimise/best_tuned.keras')
print("Evaluating model...")
result = model.evaluate(X_test_conv, y_test)
print("Accuracy of model for " + str(data) + " datapoints: ", result[1])

Evaluating model...
37/37 [==============================] - 1s 15ms/step - loss: 0.5280 - accuracy: 0.7309
Accuracy of model for 400 datapoints:  0.7308998107910156


In [45]:
# train optimised model in full

model = tf.keras.models.load_model('./models/optimise/best_tuned.keras')
model.compile(
	optimizer="adam",
	loss="binary_crossentropy",
	metrics=["accuracy"],
)

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="./models/optimise/best_tuned_full.keras",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)
model.fit(X_conv, y, 
	batch_size=32,
	epochs=100,
	callbacks=[model_checkpoint_callback],
	validation_data=(X_test_conv, y_test),
	verbose=1
)

Epoch 1/100
97/97 [==============================] - 15s 148ms/step - loss: 0.2468 - accuracy: 0.8979 - val_loss: 1.0578 - val_accuracy: 0.5407
Epoch 2/100
97/97 [==============================] - 14s 147ms/step - loss: 0.2270 - accuracy: 0.9044 - val_loss: 0.9789 - val_accuracy: 0.6392
Epoch 3/100
97/97 [==============================] - 14s 143ms/step - loss: 0.1987 - accuracy: 0.9191 - val_loss: 1.3060 - val_accuracy: 0.5586
Epoch 4/100
97/97 [==============================] - 15s 153ms/step - loss: 0.2033 - accuracy: 0.9148 - val_loss: 0.7737 - val_accuracy: 0.6715
Epoch 5/100
97/97 [==============================] - 14s 142ms/step - loss: 0.2009 - accuracy: 0.9168 - val_loss: 1.1037 - val_accuracy: 0.6443
Epoch 6/100
97/97 [==============================] - 14s 142ms/step - loss: 0.1715 - accuracy: 0.9272 - val_loss: 1.1419 - val_accuracy: 0.6553
Epoch 7/100
97/97 [==============================] - 14s 145ms/step - loss: 0.1700 - accuracy: 0.9285 - val_loss: 1.2273 - val_accuracy:

In [46]:
model = tf.keras.models.load_model('./models/optimise/best_tuned_full.keras')
model.compile(
	optimizer="adam",
	loss="binary_crossentropy",
	metrics=["accuracy"],
)

result = model.evaluate(X_test_conv, y_test)
print("Accuracy of model for " + str(data) + " datapoints: ", result[1])

37/37 [==============================] - 1s 17ms/step - loss: 1.0444 - accuracy: 0.7037
Accuracy of model for 400 datapoints:  0.7037351727485657
